In [14]:
! pip install streamlit -q

In [9]:
! wget -q -O - ipv4.icanhazip.com

35.245.179.130


In [11]:
%%writefile app.py

Writing app.py


In [15]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.179.130:8501

⠸⠼⠴your url is: https://orange-bananas-sin.loca.lt
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
/content/app.py:4: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
2024-12-21 05:46:36.415083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT

In [6]:
import os
import torch
import streamlit as st
from speechbrain.pretrained import EncoderClassifier
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
import torchaudio

# Title and Description
st.set_page_config(page_title="Speech T5 TTS App", page_icon="🎙️", layout="centered")
st.title("🎙️ SpeechT5 Text-to-Speech App")
st.markdown("""
Welcome to the **SpeechT5 Text-to-Speech** web app!
Enter text, and the fine-tuned SpeechT5 model will generate speech for you.
""")

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load models
@st.cache_resource
def load_models():
    processor = SpeechT5Processor.from_pretrained("sohail2003/pattern3")
    model = SpeechT5ForTextToSpeech.from_pretrained("sohail2003/pattern3").to(device)
    vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(device)
    speaker_model = EncoderClassifier.from_hparams(
        source="speechbrain/spkrec-xvect-voxceleb",
        run_opts={"device": device},
        savedir=os.path.join("/tmp", "speechbrain/spkrec-xvect-voxceleb"),
    )
    return processor, model, vocoder, speaker_model

processor, model, vocoder, speaker_model = load_models()

# Generate random speaker embedding
def generate_random_speaker_embedding():
    return torch.randn(1, 512).float().to(device)

# Generate speech from text
def generate_speech(text):
    speaker_embedding = generate_random_speaker_embedding()
    inputs = processor(text=text, return_tensors="pt").to(device)
    with torch.no_grad():
        speech = model.generate_speech(inputs["input_ids"], speaker_embedding, vocoder=vocoder)
    if speech.dim() == 1:
        speech = speech.unsqueeze(0)
    return speech

# Input form for user text
with st.form(key="tts_form"):
    user_text = st.text_area("Enter text to convert to speech:", placeholder="Type your text here...")
    submit_button = st.form_submit_button(label="Generate Speech 🎤")

# If user submits text
if submit_button:
    if user_text.strip():
        with st.spinner("Generating speech... 🎶"):
            speech = generate_speech(user_text.strip())
            output_file = "output.wav"
            torchaudio.save(output_file, speech.cpu(), 16000)
            st.success("Speech generated successfully!")
            st.audio(output_file, format="audio/wav")
    else:
        st.error("Please enter valid text before submitting.")


2024-12-21 05:28:16.608 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 05:28:16.627 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 05:28:16.852 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-12-21 05:28:16.863 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 05:28:16.864 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 05:28:16.868 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-21 05:28:16.890 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

DeltaGenerator(_form_data=FormData(form_id='tts_form'))

In [ ]:
y
